<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Migrate Your Virtual Workspace</h1>
    </div>
</div>

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Note</b></p>
        <p>This notebook is in preview and requires your Organization to have access to the Migrate Virtual Workspace Feature. Please reach out to us if you'd like to request access.</p>
        <p>This notebook should be run on any shared deployment (including on a Free Starter Workspace) in an Organization whose plan allow for the creation of a new dedicated deployment.</p>
        <p>To create a Free Starter Workspace navigate to <tt>Start</tt> using the left nav.</p>
    </div>
</div>

This notebook will show you how to migrate your shared deployment to a new dedicated deployment.
All the fields have default values set, but we encourage you to customize them.
The required information is:
- a new name for your new Workspace in the dedicated deployment
- the desired size of your new Workspace in the dedicated deployment (see: https://www.singlestore.com/cloud-pricing/)
- a custom name for your new workspace group
- the id of the region to create your new workspace group

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Warning</b></p>
        <p>During the migration process data writen to the shared deployment may not be moved to the new Workspace in the dedicated deployment. The shared deployment will remain available during (and after) the migration process, however any writes to the shared deployment after the migration process begins may not be reflected on the new workspace.</p>
        <p>Please account for this behaviour when migrating.</p>
        <p></p>
        <p>The migration process is done in two steps. First your data is migrated to the new deployment. Then, after the first step is completed you can migrate any jobs you had scheduled for your shared deployemnt to the new deployment. So that You can ensure that your new deployment is setup as You desire before migrating your scheduled jobs over, we advise that You do not start the second step before visiting the new deployment and checking its settings and data.</p>
        <p>While your migrated data will also remain available on your shared deployment, after the migration, the scheduled jobs will not, please account for this when migrating.</p>
    </div>
</div>

## Choose a Region for the new workspace group
Please select the Region you desire your new Workspace Group to be created on from the dropdown after the next cell.

In [1]:
import requests, os
from ipywidgets import widgets

api_url = os.environ['SINGLESTOREDB_MANAGEMENT_BASE_URL']
region_response = requests.get('{}/v1/regions'.format(api_url), headers={'Authorization': 'Bearer {}'.format(connection_password)})
regions = [[f"{r['provider']}: {r['region']}", r['regionID']] for r in region_response.json()]
region_dropdown = widgets.Dropdown(
    options = regions,
    value = regions[0][1],
    description = 'Region:',
    disabled = False,
)

display(region_dropdown)

## Choose a Name for the new workspace group
Please insert the name for your new Workspace Group in text box after the next cell.

In [2]:
from ipywidgets import widgets

workspace_group_name_text_box = widgets.Text(
    value='Migrate Group',
    placeholder='Insert a name',
    description='Name:'
)

display(workspace_group_name_text_box)

## Choose a Name for the new workspace
Please insert the name for your new Workspace in text box after the next cell.

In [3]:
from ipywidgets import widgets

workspace_name_text_box = widgets.Text(
    value='Migrated Workspace',
    placeholder='Insert a name',
    description='Name:'
)

display(workspace_name_text_box)

## Choose the size for the new workspace
Please insert the size for your new Workspace in text box after the next cell.
If you are unsure which size to choose, visit the [docs](https://www.singlestore.com/cloud-pricing/).

In [4]:
from ipywidgets import widgets

size_text_box = widgets.Text(
    value='S-00',
    placeholder='Insert a size',
    description='Size:',
    layout={'width': 'max-content'}
)

display(size_text_box)

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Warning</b></p>
        <p>During the migration process data writen to the shared deployment may not be moved to the new Workspace in the dedicated deployment. The shared deployment will remain available during (and after) the migration process, however any writes to the shared deployment after the migration process begins may not be reflected on the new workspace.</p>
        <p>Please account for this behaviour when migrating.</p>
    </div>
</div>

## Begin the migration process
Runing the next cell will start the migration process

In [5]:
import requests, os
from ipywidgets import widgets

workspace_id = os.environ['SINGLESTOREDB_VIRTUAL_WORKSPACE']
api_url = os.environ['SINGLESTOREDB_MANAGEMENT_BASE_URL']

migrate_response = requests.post('{}/v1/sharedtier/virtualWorkspaces/{}/migrate'.format(api_url, workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)}, json={
  "destinationWorkspace": {
    "name": workspace_name_text_box.value,
    "regionID": region_dropdown.value,
    "size": size_text_box.value,
    "workspaceGroupName": workspace_group_name_text_box.value
  }
})

if migrate_response.status_code == 200:
    display(widgets.HTML(f"<div class='alert alert-block alert-success'><b class='fa fa-solid fa-check-circle'></b><div><p><b>Migration Started</b></p><p>See next cell for progress</p></div></div>"))
else:
    display(widgets.HTML(f"<div class='alert alert-block alert-danger'><b class='fa fa-solid fa-exclamation-triangle'></b><div><p><b>Failed to Start Migration</b></p><p>{migrate_response.status_code} - {migrate_response.text}</p></div></div>"))

## Check the progress of the migration:
Running the next cell will give you progress on the migration

In [6]:
import requests, os, time
from ipywidgets import widgets

workspace_id = os.environ['SINGLESTOREDB_VIRTUAL_WORKSPACE']
organization_id = os.environ['SINGLESTOREDB_ORGANIZATION']
api_url = os.environ['SINGLESTOREDB_MANAGEMENT_BASE_URL']

status_response = requests.get('{}/v1/sharedtier/virtualWorkspaces/{}/migrate'.format(api_url, workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)})

while status_response.json()["migrationStatus"] not in ["Completed", "Failed"]:
    display(widgets.IntProgress(
        value=["Failed", "WaitingForWorkspaceGroup", "MigratingData", "Completed"].index(status_response.json()["migrationStatus"]),
        min=0,
        max=3,
        description='Loading:',
        orientation='horizontal'
    ))
    time.sleep(15)
    status_response = requests.get('{}/v1/sharedtier/virtualWorkspaces/{}/migrate'.format(api_url, workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)})


if status_response.json()["migrationStatus"] == "Completed":
    display(widgets.HTML('<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;"><p>The migration has completed succesfully! You can now visit your new workpace in the deployments page.</p></div></div>'))
else:
    display(widgets.HTML("<div class='alert alert-block alert-danger'><b class='fa fa-solid fa-exclamation-triangle'></b><div><p><b>Failed to Complete Migrate</b></p><p>The migration process failed, please contact support</p></div></div>"))

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Warning</b></p>
        <p>The next Step is moving the Scheduled Jobs from your shared deployement to the dedicated one.</p>
        <p>Before starting it, we advise that You visit the new deployment and checking its settings and data.</p>
        <p>While your migrated data is still available on your shared deployment, after the migration, the scheduled jobs will not be, please account for this when migrating.
    </div>
</div>

## Begin the scheduled jobs migration process
Runing the next cell will start the scheduled jobs migration process

In [7]:
import requests, os
from ipywidgets import widgets

workspace_id = os.environ['SINGLESTOREDB_VIRTUAL_WORKSPACE']
api_url = os.environ['SINGLESTOREDB_MANAGEMENT_BASE_URL']

migrate_response = requests.post('{}/v1/sharedtier/virtualWorkspaces/{}/migrateJobs'.format(api_url,workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)})

if migrate_response.status_code == 200:
    display(widgets.HTML(f"<div class='alert alert-block alert-success'><b class='fa fa-solid fa-check-circle'></b><div><p><b>Scheduled Jobs Migration Started</b></p><p>See next cell for progress</p></div></div>"))
else:
    display(widgets.HTML(f"<div class='alert alert-block alert-danger'><b class='fa fa-solid fa-exclamation-triangle'></b><div><p><b>Failed to Start Scheduled Jobs Migration</b></p><p>{migrate_response.status_code} - {migrate_response.text}</p></div></div>"))

## Check the progress of the cheduled jobs migration:
Running the next cell will give you progress on the cheduled jobs migration

In [8]:
import requests, os, time
from ipywidgets import widgets

workspace_id = os.environ['SINGLESTOREDB_VIRTUAL_WORKSPACE']
organization_id = os.environ['SINGLESTOREDB_ORGANIZATION']
api_url = os.environ['SINGLESTOREDB_MANAGEMENT_BASE_URL']

status_response = requests.get('{}/v1/sharedtier/virtualWorkspaces/{}/migrateJobs'.format(api_url, workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)})

while status_response.json()["migrationStatus"] not in ["Completed", "Failed"]:
    display(widgets.IntProgress(
        value=["Failed", "MigratingData", "Completed"].index(status_response.json()["migrationStatus"]),
        min=0,
        max=1,
        description='Loading:',
        orientation='horizontal'
    ))
    time.sleep(15)
    status_response = requests.get('{}/v1/sharedtier/virtualWorkspaces/{}/migrateJobs'.format(api_url, workspace_id), headers={'Authorization': 'Bearer {}'.format(connection_password)})


if status_response.json()["migrationStatus"] == "Completed":
    display(widgets.HTML('<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;"><p>The migration has completed succesfully! You can check the status of your scheduled jobs in the jobs page.</p></div></div>'))
else:
    display(widgets.HTML("<div class='alert alert-block alert-danger'><b class='fa fa-solid fa-exclamation-triangle'></b><div><p><b>Failed to Complete Scheduled Jobs Migration</b></p><p>The migration process failed, please contact support</p></div></div>"))

<div id="singlestore-footer" style="background-color: rgba(194, 193, 199, 0.25); height:2px; margin-bottom:10px"></div>
<div><img src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/singlestore-logo-grey.png" style="padding: 0px; margin: 0px; height: 24px"/></div>